After the previous 3 case studies,  your team has now equipped with all the three powerful skills of data science: Hacking skill, Business skill and Math skill. In this project, your team is going to make use of these skills to come up with an idea of a new business/startup based upon data science technology. Your goal is to design a better service/solution on any data you like, develop a prototype/demo and prepare a pitch for your idea.
* Your team needs to decide which business problem is important for the market you are joining in (for example, social media market, housing market, search market, etc.).
* Then design a data science approach to improve one of the current services or design a new service on any data that you choose (hopefully not Yelp dataset again).
* The solution should include all the three components of data science: 1) the business part to analyze the potential impact of your new/improved service, why the idea can make money, how much are you evaluating the company; How are you planing to persuade the sharks to invest in your business; 2) the mathematical part to formulate the problem and develop math solution; 3) the programming part to collect the data, implement the math solution, and develop the prototype/demo.


# Background about Elevator Pitch (90 seconds) and Shark Tank

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("mrSmaCo29U4") 

In [ ]:
YouTubeVideo("g31M8WmKeqg") 


**Optional Readings:** 
* LinkedIn API: https://developer.linkedin.com/docs/rest-api
* Zillow API: https://pypi.python.org/pypi/pyzillow
* Google Map API: https://developers.google.com/api-client-library/python/apis/mapsengine/v1?hl=en
* More APIs: https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition


** Python libraries you may want to use:**
* Scikit-learn (http://scikit-learn.org): machine learning tools in Python.

** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

#Problem 1: the Business Part (20 points)
 As a group, learn about the data science related business and research about the current markets: such as search, social media, advertisement, recommendation and so on.
Pick one of the markets for further consideration, and design a new service  which you believe to be important in the market. 
Define precisely in the report and briefly in the cells below, what is the business problem that your team wants to solve.
Why the problem is important to solve? 
Why you believe you could make a big difference with data science technology.
How are you planing to persuade the investors to buy in your idea.

**Please describe here *briefly*  (please edit this cell)**

1) Your business problem to solve:



2) Why the problem is important to solve? 


3) What is your idea to solve the problem? 


4) What differences you could make with your data science approach?


5) Why do you believe the idea deserves the investment of the "sharks"?






#Problem 2: The Math Part (20 points)

Define the business problem as a math problem and design a math solution to the problem.


**Insert your answers here**


1) Problem formulation in Math:




2) Math Solution:






3) Implementation of the Solution




#Problem 3: The Hacking Part  (20 points)


* Data Collection
* Implement a small Demo/Prototype/experiment result figures for the "product" of your data science company. You could use this demo during the Pitch

**Data Collection**:
This part is about pre-processing. Such like some parameter settings, street information gathering and save the data into "data" file.

In [1]:
#===========================all imports, class and function defination!=======================
from pyzillow.pyzillow import ZillowResults,ZillowWrapper, GetDeepSearchResults
import json,pickle,os

def dumps(data):
    with open("data",'wb') as out:
        pickle.dump(data, out, pickle.HIGHEST_PROTOCOL)

def load():
    if not os.path.exists("data"):
        dumps([None,None]) 
    with open("data","rb") as files:
        data=pickle.load(files)
        return data
        
def dictUpdate(old,new):
    if len(new.keys())==0: return
    key=new.keys()[0]
    if old.has_key(key):
        if old[key].__class__!=[].__class__:
            old[key]=[old[key]]
        old[key].append(new[key])
    else:
        old.update(new)

def JRes(elem):
        d=dict()
        tag=elem.tag.split("}")[-1]
        if elem.text:
            d[tag]=elem.text
        children=elem.getchildren()
        if children:
            li=map(JRes, children)
            content_dict={tag:[]}
            newli={}
            for i in li:
                if len(i)==1 and i.has_key(tag):
                    content_dict[tag].append(i[tag])
                else:
                    dictUpdate(newli,i)
            if not content_dict[tag]:
                content_dict={}
            if(len(newli)>0):
                newli.update(content_dict)
            else:
                newli=content_dict
            d[tag]=newli
        return d


API_KEY = "X1-ZWz19zavpa7evf_4pyss"#"X1-ZWz19z1o83a70r_1acr8" Changing it! "X1-ZWz19zavpa7evf_4pyss" is mine
         
zillow_data = ZillowWrapper(API_KEY) 



#======================================fetch all region in MA======================================

#========set_parameter_here======
#Select an api from list on http://www.zillow.com/howto/api/APIOverview.htm 
api_url="http://www.zillow.com/webservice/GetRegionChildren.htm"
#specify parameters listed in table
params={
    "zws-id":API_KEY,   
    "state":"MA"}
regList=['Worcester']

data=load()
if data[0]==None:
    #=========fetching===============
    #Get result in MA
    res_lv0=JRes(zillow_data.get_data(api_url,params))
    #Get county region id list
    res_lv1=res_lv0["regionchildren"]["response"]["list"]["region"]
    res_lv1_filtered=[i['id'] for i in res_lv1 if i['name'] in regList]
    #Get to next level
    master={}
    for id in res_lv1_filtered:
        res_lv2=JRes(zillow_data.get_data(api_url,{"zws-id":API_KEY,"regionId":id}))['regionchildren']['response']
        master[res_lv2["region"]["county"]]=res_lv2["list"]["region"]
    #level 3 takes a little longer
    res_lv3=[
        JRes(zillow_data.get_data(api_url,{"zws-id":API_KEY,"regionId":id}))['regionchildren']['response']["list"]
        for id in [j['id'] for i in regList for j in master[i]]]
    #zip codes
    tmp=[]
    for i in [i['region'] for i in res_lv3 if i.has_key('region')]:
        if i.__class__=={}.__class__:
            tmp.append(i)
        else:
            tmp+=i
    zips=[i['name'] for i in tmp]
    data[0]=zips
else: pass
dumps(data)


# ==================================Get residual addresses -_-!==============================
# ===it is from www.melissadata.com
import urllib2,re
from bs4 import BeautifulSoup

def getStreet(zipcode):
    query = zipcode
    url = 'http://www.melissadata.com/lookups/zipstreet.asp?InData=' + query
    req = urllib2.Request(url)
    con = urllib2.urlopen( req)
    doc = con.read()
    con.close()
    matchObj = re.finditer( r'(?<=zipstreet\.asp\?Step5=\d{5,5}\&Name=)\w+(\+\w+)*', doc)
    streets=[]
    for m in matchObj:
        streets.append(m.group())
    return streets,doc 
        
data=load()
if data[1]==None:
    streets={}   
    for zipcode in data[0]:
        print zipcode
        street,doc=getStreet(zipcode)
        streets[zipcode]=street
    for i in streets:
        #print i,streets[i]
        for j in range(len(streets[i])):
            streets[i][j]=streets[i][j].replace("+"," ")
    data[1]=streets
    
dumps(data)
data[1]['01609']


['ACADEMY',
 'ALGONQUIN',
 'ARDMORE',
 'ARGYLE',
 'ASHLAND',
 'AUSTIN',
 'AVALON',
 'AYLESBURY',
 'BANCROFT TOWER',
 'BARROWS',
 'BARRY',
 'BEECHMONT',
 'BEL AIR',
 'BELLEVUE',
 'BERKSHIRE',
 'BIRCHWOOD',
 'BLACKTHORN',
 'BLACKWELL',
 'BLOSSOM',
 'BOWDOIN',
 'BOYNTON',
 'BREWER',
 'BRIGHAM',
 'BROMPTON',
 'BROOK HILL',
 'BROOKSHIRE',
 'BROWNING',
 'BURGESS',
 'BUTTERNUT HILL',
 'CARTER',
 'CEDAR',
 'CHANDLER',
 'CHATHAM',
 'CHESTNUT',
 'CHESTNUT HILL',
 'CHILTERN HILL',
 'CLINTON',
 'COLUMBINE',
 'CONGRESS',
 'COTTAGE',
 'CROWN',
 'DARA',
 'DARENESS',
 'DAYTON',
 'DEAN',
 'DENNISON',
 'DENNY',
 'DEWEY',
 'DICK',
 'DIX',
 'DONNA',
 'DOVER',
 'DRURY',
 'DUBIEL',
 'EDEN',
 'EINHORN',
 'ELBRIDGE',
 'ELLIS',
 'ELM',
 'ESSEX',
 'FARNUM',
 'FENIMORE',
 'FLAGG',
 'FLOWER HILL',
 'FOREST',
 'FOREST HILL',
 'FRUIT',
 'GEORGE',
 'GLENBROOK',
 'GOULDING',
 'GREEN VIEW',
 'GREENSIDE',
 'GREENWICH',
 'GROSS',
 'HACKFELD',
 'HAMPDEN',
 'HANCOCK HILL',
 'HARVARD',
 'HAWLEY',
 'HAZELWOOD',
 'HICKORY',


**More collecting!**
This part is for collecting data from zillow. Since there is a limited times of accessing zillow API, which is 1000 times per day. So we create this cell to allow us to fetch data with multiple computers.

In [2]:
#get property details

def getZpid(addr,zipcode):
    #========set_parameter_here======
    api_url="http://www.zillow.com/webservice/GetDeepSearchResults.htm"
    #specify parameters listed in table
    params={"zws-id":API_KEY,"address":addr,"citystatezip":zipcode}

    #=========fetching===============
    #Get result in MA
    
    return JRes(zillow_data.get_data(api_url,params))["searchresults"]['response']#['results']['result']
zipPool=["01609",]
streetPool=data[1]['01609']#################### ADD ST NAME HERE listed on above cell #####################
data=load()
if len(data)==2:
    zpidList={}
    data.append(None)
elif len(data)==3:
    zpidList=data[2]    

for zipcode in data[1]:
    #print 111#######
    if not zipcode in zipPool:
        continue#Just retrieve zipcode in Pool
    if not zpidList.has_key(zipcode): 
        zpidList[zipcode]={"content":{},"Done":False}
    if zpidList[zipcode]["Done"]:continue
    else:
        for street in data[1][zipcode]:
            #print 222#######
            if not street in streetPool:
                continue#Just retrieve streets in Pool
            print street,zipcode
            if not zpidList[zipcode]["content"].has_key(street):
                zpidList[zipcode]["content"][street]={"content":{},"Done":False}
            if zpidList[zipcode]["content"][street]["Done"]:
                continue
            else:
                fail_times=0
                cont_fail_times=0
                for i in range(1,999):
                    #print 333#######
                    if fail_times>200 or cont_fail_times>50 and i>100 : 
                        print i,fail_times,cont_fail_times
                        zpidList[zipcode]["content"][street]["Done"]=True
                        break
                    if zpidList[zipcode]["content"][street]["content"].has_key(i) and zpidList[zipcode]["content"][street]["content"][i] :continue
                    try:
                        zpidList[zipcode]["content"][street]["content"][i]=getZpid(str(i)+" "+street,zipcode)
                        print "success! ",str(i)+" "+street,zipcode
                        cont_fail_times=0
                        data[2]=zpidList
                        dumps(data)
                    except Exception as err:
                        if err.message==7: 
                            print "OUT OF TIMES"
                            break
                        zpidList[zipcode]["content"][street]["content"][i]=None
                        print "Fails!   ",str(i)+" "+street,zipcode,err.message
                        fail_times+=1
                        cont_fail_times+=1
            
    #zpidList[zipcode]["Done"]=True
data[2]=zpidList
dumps(data)


ACADEMY 01609
ALGONQUIN 01609
ARDMORE 01609
ARGYLE 01609
ASHLAND 01609
AUSTIN 01609
AVALON 01609
AYLESBURY 01609
BANCROFT TOWER 01609
BARROWS 01609
BARRY 01609
BEECHMONT 01609
BEL AIR 01609
BELLEVUE 01609
BERKSHIRE 01609
BIRCHWOOD 01609
BLACKTHORN 01609
BLACKWELL 01609
BLOSSOM 01609
BOWDOIN 01609
BOYNTON 01609
BREWER 01609
BRIGHAM 01609
BROMPTON 01609
BROOK HILL 01609
BROOKSHIRE 01609
BROWNING 01609
BURGESS 01609
BUTTERNUT HILL 01609
CARTER 01609
CEDAR 01609
CHANDLER 01609
CHATHAM 01609
CHESTNUT 01609
CHESTNUT HILL 01609
CHILTERN HILL 01609
CLINTON 01609
COLUMBINE 01609
CONGRESS 01609
COTTAGE 01609
CROWN 01609
DARA 01609
DARENESS 01609
DAYTON 01609
DEAN 01609
DENNISON 01609
DENNY 01609
DEWEY 01609
DICK 01609
DIX 01609
DONNA 01609
DOVER 01609
DRURY 01609
DUBIEL 01609
EDEN 01609
EINHORN 01609
ELBRIDGE 01609
ELLIS 01609
ELM 01609
ESSEX 01609
FARNUM 01609
FENIMORE 01609
FLAGG 01609
FLOWER HILL 01609
FOREST 01609
FOREST HILL 01609
FRUIT 01609
GEORGE 01609
GLENBROOK 01609
GOULDING 01609
GREE

Your Part,  Saber. Two functions help you access data I collected.   
The problem is Zillow just provides the house value estimate. Maybe you can work on that, "Zestimate".   
Other problem is, since zillow api does not allow me access it over 1000 times, so just get the housees **ONLY ON HIGHLAND ST**. If you need more I can keep working on it and try use different computers to get more data.

In [15]:
#Mortgate provided by zillow
def getMortgate(state=None):
    url='http://www.zillow.com/webservice/GetRateSummary.htm'
    return JRes(zillow_data.get_data(url,{"zws-id":API_KEY,'state':state}))  

#Just a example saber. You can mod this func anyway, data is in "tmp" variable. Just get want you what
def itsYourDataSaber():
    tmp=load()[2].copy()
    zdata={}
    for i in tmp:
        for j in tmp[i]["content"]:
            for k,v in tmp[i]["content"][j]["content"].iteritems():
                if v:
                    t=v["results"]["result"]
                    if t.__class__!=[].__class__: t=[t]
                    zdata[str(k)+" "+j+", "+i]=t
    return zdata
#This is example that How to access data

for i,v in itsYourDataSaber().iteritems():
    for j in v:
        if i=="226 HIGHLAND, 01609":
            print i
            for key in j.keys():
                if key in ["links",'address',"localRealEstate"]: continue
                print "  |",key,":",j[key]
                
print "\nHouses Number is",len(itsYourDataSaber())               

print "\n\nhttp://www.zillow.com/howto/api/GetRateSummary.htm"                
getMortgate("MA") 
#itsYourDataSaber()

226 HIGHLAND, 01609
  | totalRooms : 21
  | lotSizeSqFt : 5489
  | finishedSqFt : 4755
  | taxAssessmentYear : 2015
  | zestimate : {'valueChange': '-4219', 'amount': '288041', 'percentile': '0', 'valuationRange': {'high': '305323', 'low': '264998'}, 'last-updated': '12/01/2015'}
  | bathrooms : 3.0
  | bedrooms : 9
  | FIPScounty : 25027
  | taxAssessment : 203400.0
  | yearBuilt : 1905
  | useCode : MultiFamily2To4
  | zpid : 56735656

Houses Number is 4360


http://www.zillow.com/howto/api/GetRateSummary.htm


{'rateSummary': {'message': {'code': '0',
   'text': 'Request successfully processed'},
  'response': {'lastWeek': {'rate': ['3.8', '2.98', '3.04']},
   'today': {'rate': ['3.86', '3.03', '3.09']}}}}

*------------------------

#Bonus Question: Prepare a 90 second Pitch and *present* it in the class (10 points)

* Prepare the slides for the Pitch (5 points)
* Present it in the class before your 10-minute talk (5 points).



*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File** (60 points): Save this IPython notebook.


* **PPT Slides** (20 points): please prepare PPT slides (for 10 minutes' talk) to present about the case study . 

* ** Report** (20 points): please prepare a report (less than 10 pages) to report what you did in this project.

     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.
